In [ ]:
#download only for colab
#!apt-get update
!apt-get install libsdl2-gfx-dev libsdl2-ttf-dev

# 2.8 and 2.9 binary is the same, so we use 2.8 .so file
!git clone -b v2.9 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 python3 -m pip install .

In [ ]:
#used for colab only 
import gfootball.env as football_env
env = football_env.create_environment(env_name="academy_empty_goal_close", stacked=False, logdir='/tmp/football', write_goal_dumps=False, write_full_episode_dumps=False, render=False)
state = env.reset()
#print(state)

In [1]:
#main for any type of device
import gfootball.env as football_env
env = football_env.create_environment(env_name='academy_empty_goal', representation='pixels', render=True)

====================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [ ]:
!pip install h5py pyyaml

In [27]:
#lib
import tensorflow as tf
import keras
import numpy as np
import gym
import os
import sys
from keras.models import Model
from keras.layers import Dense,Input,Flatten
from keras import backend as k
from tensorflow.keras.optimizers import Adam
from keras.applications.mobilenet_v2 import MobileNetV2 as mb
from keras.callbacks import ModelCheckpoint as mc

In [3]:
#constants
gamma         = 0.99
lamda         = 0.95
clip_value    = 0.2
entropy_loss  = 0.001
lr1           = 3e-4
lr2           = 1e-4
state_size    = env.observation_space.shape
action_size   = env.action_space.n
print(state_size,action_size)

(72, 96, 3) 19


In [4]:
#creating Actor
def Actor(): 
  x_input = Input(state_size)
  #prob = Input(shape = (1,action_size,))
  #advantages = Input(shape = (1,1,))
  #reward = Input(shape = (1,1,))
  #values = Input(shape =(1,1,))
  feature = mb(include_top = False , weights = "imagenet")
  for layers in feature.layers:
    layers.trainable = False
  x = Flatten(name ="flat")(feature(x_input))
  x1 = Dense(1024 ,activation = "relu")(x)
  x2 = Dense(512 ,activation ="relu")(x1)
  x3 = Dense(256,activation = "relu")(x2)
  policy = Dense(action_size , activation = "softmax")(x1)
  model =Model(inputs  = [x_input] ,outputs = policy)
  model.compile(optimizer = Adam(lr = 1e-10) ,loss = ppo_loss)
  model.summary()
  return model

In [5]:
#ppo_loss from scratch
def ppo_loss(y_true,y_pred):
   advantages, predictions, actions = y_true[:, :1], y_true[:, 1:1+action_size], y_true[:, 1+action_size:]
   prob = actions *y_pred
   old_prob =actions * predictions
   prob  = k.clip(prob ,1e-8,1)
   old_prob  = k.clip(old_prob ,1e-8,1)
   ratio = k.exp(k.log(prob) -k.log(old_prob))
   clip  = k.clip(ratio ,1-clip_value ,1+clip_value)
   s1    = ratio * advantages
   s2    = clip * advantages
   actor_loss = -k.mean(k.minimum(s1,s2))
   ent = -y_pred *k.log(y_pred)
   entropy = entropy_loss * k.mean(ent)
   loss = actor_loss - entropy
   return loss

In [6]:
#creating critic
def Critic(): 
  x_input = Input(state_size)
  #prob = Input(shape = (1,action_size,))
  #advantages = Input(shape = (1,1,))
  #reward = Input(shape = (1,1,))
  #values = Input(shape =(1,1,))
  feature = mb(include_top = False , weights = "imagenet")
  for layers in feature.layers:
    layers.trainable = False
  x = Flatten(name ="flat")(feature(x_input))
  x1 = Dense(1024 ,activation = "relu")(x)
  x2 = Dense(512 ,activation ="relu")(x1)
  x3 = Dense(256,activation = "relu")(x2)
  policy = Dense(1 , activation = None)(x1)
  model =Model(inputs  = [x_input] ,outputs = policy)
  model.compile(optimizer = Adam(lr = 1e-4) ,loss = "mse")
  model.summary()
  return model

In [30]:
path ="football/cp.ckpt"
dir =os.path.dirname(path)
call =mc(path ,save_weights_only=True,verbose =0)

In [7]:
#advantage function
def get_advantages(values,next_values ,masks, rewards):
    returns = []
    gae = 0
    for i in reversed(range(len(rewards))):
        delta = rewards[i] + gamma * next_values[i]* masks[i] - values[i]
        gae = delta + gamma * lamda * masks[i] * gae
        returns.insert(0, gae + values[i])

    adv = np.array(returns) -values
    return returns, (adv - np.mean(adv)) / (np.std(adv) + 1e-10)

In [17]:
#variable
states ,next_states,masks ,rewards,values,next_values , actions,predictions,onehots,averages =[],[],[],[],[],[],[],[],[],[]

In [31]:
class Agent:
  def __init__(self):
    self.critic = Critic()
    self.actor = Actor()
    self.episode = 0
    self.max = 10000000000
    self.Actor_name = "Actor.h5"
    self.Critic_name = "critic.h5"
  def Act(self,state):
    #frist we create  predictions, after action
    pred = self.actor.predict(state)[0]
    action = np.random.choice(action_size , p = pred)
    #coding onehot action
    onehot =np.zeros(action_size)
    onehot[action] = 1
    return pred , action,onehot
  def replay(self,rewards,masks,states,next_states,actions,predictions):
    values = self.critic.predict(states)
    next_values = self.critic.predict(next_states)
    target ,advantages = get_advantages(values,next_values ,masks, rewards)
    predictions = np.vstack(predictions)
    actions = np.vstack(actions)
    yt = np.hstack([advantages,predictions,actions])
    self.actor.fit(states , yt,epochs=10,verbose = 0,callbacks = [call])
    self.critic.fit([states,values] , target , epochs =10,verbose=0,callback = [call])
  def load(self):
    self.actor.load_weights(self.Actor_name)
    self.critic.load_weights(self.Critic_name)
  def save(self):
    self.actor.save_weights(self.Actor_name)
    self.critic.save_weights(self.Critic_name)
  def train(self):
    done = False
    score = 0
    state =env.reset()
    state1 = np.expand_dims(state , 0)
    while True:
      #kdnvkjdv
      while not done:
        for e in range(self.max):
            pred , action ,onehot =self.Act(state1)
            next_state ,reward, done , info =env.step(action)
            ns = np.expand_dims(next_state , 0)
            actions.append(action)
            predictions.append(pred)
            states.append(state)
            next_states.append(ns)
            onehots.append(onehot)
            rewards.append(reward)
            mask = not done
            masks.append(mask)
            #self.replay(rewards,masks,states,next_states,actions,predictions)
            state = ns
            score += reward
            if done:
              self.episode +=1
              print("episode:",e ,"action:",action , "reward:",score)
              done = False
              score = 0
              state =env.reset()
              state = np.expand_dims(state , 0)
              average = reward.mean()
              averages.append(average)
              if average >reward:
                self.actor.save(self.Actor_name)
                self.critic.save(self.Critic_name)
        self.replay(rewards,masks,states,next_states,actions,predictions)
        if self.episode >=self.max:
          break
      env.close()

In [ ]:
if __name__ == "__main__":
  agent = Agent()
  agent.train()

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, 72, 96, 3)]       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, None, None, 1280)  2257984  
 ional)                                                          
                                                                 
 flat (Flatten)              (None, 7680)              0         
                                                                 
 dense_72 (Dense)            (None, 1024)              7865344   
                                                                 
 dense_75 (Dense)            (None, 1)                 1025      
                                                                 
Total params: 10,124,353
Trainable params: 7,866,369
Non-trainable params: 2,257,984
_______________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_39 (InputLayer)       [(None, 72, 96, 3)]       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, None, None, 1280)  2257984  
 ional)                                                          
                                                                 
 flat (Flatten)              (None, 7680)              0         
                                                                 
 dense_76 (Dense)            (None, 1024)              7865344   
                                                                 
 dense_79 (Dense)            (None, 19)                19475     
                                                                 
Total params: 10,142,803
Trainable params: 7,884,819
Non-trainable params: 2,257,984
_______________________________________

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
